## Exercícios

> Retirados de [learn-python: sqlalchemy_orm-questions](https://aviadr1.github.io/learn-advanced-python/11_db_access/exercise/sqlalchemy_orm-questions.html).

#### Q1.

Baixa e extraia o arquivo compactado com o banco de dados [Chinook database](https://www.sqlitetutorial.net/sqlite-sample-database/). Salve o arquivo `chinook.db` na mesma pasta deste script.
* Link para baixar: http://www.sqlitetutorial.net/wp-content/uploads/2018/03/chinook.zip

<img width=500 src=https://www.sqlitetutorial.net/wp-content/uploads/2015/11/sqlite-sample-database-color.jpg>


#### Q2.

Leia o código e os comentários das células a seguir para entender como acessamos os modelos ORM de um banco já existente.

In [3]:
from sqlalchemy import create_engine, text, MetaData,select
from sqlalchemy.orm import Session

engine = create_engine("sqlite+pysqlite:///chinook.db", echo=False)

### extrai as classes da base de dados Chinook
metadata = MetaData()
metadata.reflect(engine)

# O metadata tem informações sobre as tabelas
# que serão usadas para criar os modelos ORM
for table_name, table in metadata.tables.items():
    print(table_name)
    print(table.columns.keys())
    print(table.columns.items())
    print('-'*25)

### configura o objeto Base mapeando os modelos ORM das tabelas
from sqlalchemy.ext.automap import automap_base
Base = automap_base(metadata=metadata)
Base.prepare()

# o objeto Base tem os modelos ORM que podemos usar
# para manipular o banco de dados
print(Base.classes.items())

albums
['AlbumId', 'Title', 'ArtistId']
[('AlbumId', Column('AlbumId', INTEGER(), table=<albums>, primary_key=True, nullable=False)), ('Title', Column('Title', NVARCHAR(length=160), table=<albums>, nullable=False)), ('ArtistId', Column('ArtistId', INTEGER(), ForeignKey('artists.ArtistId'), table=<albums>, nullable=False))]
-------------------------
artists
['ArtistId', 'Name']
[('ArtistId', Column('ArtistId', INTEGER(), table=<artists>, primary_key=True, nullable=False)), ('Name', Column('Name', NVARCHAR(length=120), table=<artists>))]
-------------------------
customers
['CustomerId', 'FirstName', 'LastName', 'Company', 'Address', 'City', 'State', 'Country', 'PostalCode', 'Phone', 'Fax', 'Email', 'SupportRepId']
[('CustomerId', Column('CustomerId', INTEGER(), table=<customers>, primary_key=True, nullable=False)), ('FirstName', Column('FirstName', NVARCHAR(length=40), table=<customers>, nullable=False)), ('LastName', Column('LastName', NVARCHAR(length=20), table=<customers>, nullable=F

In [4]:
# A seguir um exemplo de query na tabela Albums
# usamos o objeto Base para acessar cada modelo ORM.

session = Session(engine)
res = session.scalars(select(Base.classes.albums))
first_album = res.first()
print(first_album.AlbumId, first_album.Title)

1 For Those About To Rock We Salute You


#### Q3. 
Com base nos códigos anteriores realize as operações solicitadas nas células a seguir:


In [10]:
### Imprima os três primeiros registros da tabela tracks
session = Session(engine)
result = session.scalars(select(Base.classes.tracks).limit(3))
for track in result:
    print(track.TrackId, track.Name)


1 For Those About To Rock (We Salute You)
2 Balls to the Wall
3 Fast As a Shark


In [11]:
### Imprima o nome da faixa e o título do álbum das primeiras 20 faixas na tabela tracks.
session = Session(engine)
Track = Base.classes.tracks
Album = Base.classes.albums

stmt = select(Track, Album).join(Album, Track.AlbumId == Album.AlbumId).limit(20)
result = session.execute(stmt)

for track, album in result:
    print(track.Name, album.Title)


For Those About To Rock (We Salute You) For Those About To Rock We Salute You
Put The Finger On You For Those About To Rock We Salute You
Let's Get It Up For Those About To Rock We Salute You
Inject The Venom For Those About To Rock We Salute You
Snowballed For Those About To Rock We Salute You
Evil Walks For Those About To Rock We Salute You
C.O.D. For Those About To Rock We Salute You
Breaking The Rules For Those About To Rock We Salute You
Night Of The Long Knives For Those About To Rock We Salute You
Spellbound For Those About To Rock We Salute You
Balls to the Wall Balls to the Wall
Fast As a Shark Restless and Wild
Restless and Wild Restless and Wild
Princess of the Dawn Restless and Wild
Go Down Let There Be Rock
Dog Eat Dog Let There Be Rock
Let There Be Rock Let There Be Rock
Bad Boy Boogie Let There Be Rock
Problem Child Let There Be Rock
Overdose Let There Be Rock


In [12]:
### Imprima as 10 primeiras vendas de faixas da tabela invoice_items
### Para essas 10 primeiras vendas, imprima os nomes das faixas vendidas e a quantidade vendida.
session = Session(engine)
Track = Base.classes.tracks
InvoiceItem = Base.classes.invoice_items

stmt = (
    select(InvoiceItem, Track)
    .join(Track, InvoiceItem.TrackId == Track.TrackId)
    .limit(10)
)
result = session.execute(stmt)

for invoice_item, track in result:
    print(track.Name, invoice_item.Quantity)



Balls to the Wall 1
Restless and Wild 1
Put The Finger On You 1
Inject The Venom 1
Evil Walks 1
Breaking The Rules 1
Dog Eat Dog 1
Overdose 1
Love In An Elevator 1
Janie's Got A Gun 1


In [13]:
### Imprima os nomes das 10 faixas mais vendidas e quantas vezes foram vendidas.
session = Session(engine)
Track = Base.classes.tracks
InvoiceItem = Base.classes.invoice_items

from sqlalchemy import func, desc

stmt = (
    select(Track.Name, func.sum(InvoiceItem.Quantity).label('total_sales'))
    .join(InvoiceItem, Track.TrackId == InvoiceItem.TrackId)
    .group_by(Track.TrackId)
    .order_by(desc('total_sales'))
    .limit(10)
)
result = session.execute(stmt)

for name, total_sales in result:
    print(name, total_sales)


Balls to the Wall 2
Inject The Venom 2
Snowballed 2
Overdose 2
Deuces Are Wild 2
Not The Doctor 2
Por Causa De Você 2
Welcome Home (Sanitarium) 2
Snowblind 2
Cornucopia 2


In [14]:
### Quem são os 10 artistas que mais venderam?
### dica: você precisa juntar as tabelas invoice_items, tracks, albums e artists
session = Session(engine)
Artist = Base.classes.artists
Album = Base.classes.albums
Track = Base.classes.tracks
InvoiceItem = Base.classes.invoice_items

from sqlalchemy import func, desc

stmt = (
    select(Artist.Name, func.sum(InvoiceItem.Quantity).label('total_sales'))
    .join(Album, Album.ArtistId == Artist.ArtistId)
    .join(Track, Track.AlbumId == Album.AlbumId)
    .join(InvoiceItem, InvoiceItem.TrackId == Track.TrackId)
    .group_by(Artist.ArtistId)
    .order_by(desc('total_sales'))
    .limit(10)
)

result = session.execute(stmt)

for artist_name, total_sales in result:
    print(artist_name, total_sales)


Iron Maiden 140
U2 107
Metallica 91
Led Zeppelin 87
Os Paralamas Do Sucesso 45
Deep Purple 44
Faith No More 42
Lost 41
Eric Clapton 40
R.E.M. 39
